In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from io import BytesIO
import requests
import pathlib
import os

I run the scripts zec_news_data_pull.py and zec_price_data_pull.py daily using the following .plist scripts:


### Pull price daily
<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE plist PUBLIC "-//Apple//DTD PLIST 1.0//EN"
 "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
<plist version="1.0">
<dict>

    <key>Label</key>
    <string>com.maria.data_pull_daily</string>

    <key>ProgramArguments</key>
    <array>
        <string>/opt/anaconda3/envs/myenv/bin/python</string>
        <string>/Users/maria/springboard/Project/data_pull_daily.py</string>
    </array>

    <key>WorkingDirectory</key>
    <string>/Users/maria/springboard/Project/</string>

    <key>EnvironmentVariables</key>
    <dict>
        <key>PATH</key>
        <string>/opt/anaconda3/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin</string>
    </dict>

    <key>StandardOutPath</key>
    <string>/Users/maria/data_pull_daily.log</string>

    <key>StandardErrorPath</key>
    <string>/Users/maria/data_pull_daily_error.log</string>

    <key>RunAtLoad</key>
    <true/>
    
    <!-- Schedule daily run at 08:00 -->
    <key>StartCalendarInterval</key>
    <dict>
        <key>Hour</key>
        <integer>6</integer>
        <key>Minute</key>
        <integer>22</integer>
    </dict>
</dict>
</plist>

### Pull news daily
<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE plist PUBLIC "-//Apple//DTD PLIST 1.0//EN"
 "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
<plist version="1.0">
<dict>

    <key>Label</key>
    <string>com.maria.data_pullnews_daily</string>

    <key>ProgramArguments</key>
    <array>
        <string>/opt/anaconda3/envs/myenv/bin/python</string>
        <string>/Users/maria/springboard/Project/import_requests_newsdata.py</string>
    </array>

    <key>WorkingDirectory</key>
    <string>/Users/maria/springboard/Project/</string>

    <key>EnvironmentVariables</key>
    <dict>
        <key>PATH</key>
        <string>/opt/anaconda3/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin</string>
    </dict>

    <key>StandardOutPath</key>
    <string>/Users/maria/data_pullnews_daily.log</string>

    <key>StandardErrorPath</key>
    <string>/Users/maria/data_pullnews_daily_error.log</string>

    <key>RunAtLoad</key>
    <true/>
    
    <!-- Schedule daily run at 08:00 -->
    <key>StartCalendarInterval</key>
    <dict>
        <key>Hour</key>
        <integer>6</integer>
        <key>Minute</key>
        <integer>25</integer>
    </dict>
</dict>
</plist>



I collect all stored files and consolidate into a single news or prices .csv files

### Read all zec files

In [2]:
directory = "/Users/maria/springboard/Project/data/"
files = [f for f in os.listdir(directory) if f.startswith("test_zec_daily_5min")]
df = pd.concat((pd.read_csv(directory+f, usecols=['timestamp', 'price', 'market_caps', 'total_volumes']) for f in files), ignore_index=True)
df.to_csv('data/all_zec.csv', index=False)

### Read News files

In [3]:
directory = "/Users/maria/springboard/Project/data/news/"
#ongoing news
files = [f for f in os.listdir(directory) if f.startswith("newsdata")]
df_1 = pd.concat((pd.read_csv(directory+f) for f in files), ignore_index=True)
#historical news
df = pd.read_csv('data/news/newsdata_historical.csv', date_format='pubDate')
#reshape and merge
#df_1 = df_1.rename(columns = {:'pubDate'})
df_1 = df_1.drop(['category','country','date'], axis = 1)
col_list = df_1.columns

df_all= pd.concat([df[col_list], df_1],ignore_index=True)
df_all.drop_duplicates(subset='article_id',inplace=True)
df_all.to_csv('data/news/all_news.csv', index=False)